In [7]:
import os
import sys
import pandas as pd

package_path = '/Users/flaminia/Documents/work/ProMCDA'

if package_path not in sys.path:
    sys.path.append(package_path)

try:
    from mcda.models.ProMCDA import ProMCDA
    print("Import successful!")
except ModuleNotFoundError as e:
    print(f"ModuleNotFoundError: {e}")

from mcda.configuration.enums import NormalizationFunctions, AggregationFunctions, OutputColumnNames4Sensitivity, NormalizationNames4Sensitivity, PDFType

Import successful!


In [8]:
def setUp():
    
    # Mock input data for testing
    input_matrix_without_uncertainty = pd.DataFrame({
        'Criteria 1': [0.5, 0.2, 0.8],
        'Criteria 2': [0.3, 0.6, 0.1]
    }, index=['A', 'B', 'C'])

    input_matrix_with_uncertainty = pd.DataFrame({
    'alternatives': ['alt1', 'alt2', 'alt3', 'alt4'],
    'ind1_min': [-15.20, -12.40, 10.60, -39.70],
    'ind1_max': [8.20, 8.70, 2.00, 14.00],
    'ind2': [0.04, 0.05, 0.11, 0.01],
    'ind3_average': [24.50, 24.50, 14.00, 26.50],
    'ind3_std': [6.20, 4.80, 0.60, 4.41],
    'ind4_average': [-15.20, -12.40, 1.60, -39.70],
    'ind4_std': [8.20, 8.70, 2.00, 14.00],
    'ind5': [0.04, 0.05, 0.11, 0.01],
    'ind6_average': [24.50, 24.50, 14.00, 26.50],
    'ind6_std': [6.20, 4.80, 0.60, 4.41]
    })
   
    input_matrix_with_uncertainty.set_index('alternatives', inplace=True)

    polarity = ('+', '-')

    robustness_weights = False
    robustness_indicators = True

    marginal_distributions = [PDFType.NORMAL, PDFType.NORMAL]

    num_runs = 5
    num_cores = 1

    output_path = 'mock_output/'

    # Return the setup parameters as a dictionary
    return {
        'input_matrix': input_matrix_with_uncertainty, # Decide what type of input matrix
        'polarity': polarity,
        'robustness_weights': robustness_weights,
        'robustness_indicators': robustness_indicators,
        'marginal_distributions': marginal_distributions,
        'num_runs': num_runs,
        'num_cores': num_cores,
        'output_path': output_path
    }

# Run the setup and store parameters in a variable
setup_parameters = setUp()

# Check the setup parameters
setup_parameters

{'input_matrix':               ind1_min  ind1_max  ind2  ind3_average  ind3_std  ind4_average  \
 alternatives                                                                   
 alt1             -15.2       8.2  0.04          24.5      6.20         -15.2   
 alt2             -12.4       8.7  0.05          24.5      4.80         -12.4   
 alt3              10.6       2.0  0.11          14.0      0.60           1.6   
 alt4             -39.7      14.0  0.01          26.5      4.41         -39.7   
 
               ind4_std  ind5  ind6_average  ind6_std  
 alternatives                                          
 alt1               8.2  0.04          24.5      6.20  
 alt2               8.7  0.05          24.5      4.80  
 alt3               2.0  0.11          14.0      0.60  
 alt4              14.0  0.01          26.5      4.41  ,
 'polarity': ('+', '-'),
 'robustness_weights': False,
 'robustness_indicators': True,
 'marginal_distributions': [<PDFType.NORMAL: 'normal'>,
  <PDFType.NORMA

In [9]:
promcda = ProMCDA(
    input_matrix=setup_parameters['input_matrix'],
    polarity=setup_parameters['polarity'],
    robustness_weights=setup_parameters['robustness_weights'],
    robustness_indicators=setup_parameters['robustness_indicators'],
    marginal_distributions=setup_parameters['marginal_distributions'],
    num_runs=setup_parameters['num_runs'],
    num_cores=setup_parameters['num_cores'],
    #output_path=setup_parameters['output_path']
)

In [10]:
promcda.normalize()

INFO: 2024-11-27 15:37:19,707 - ProMCDA - Alternatives are ['alt1', 'alt2', 'alt3', 'alt4']
INFO: 2024-11-27 15:37:19,713 - ProMCDA - There is a problem with the parameters given in the input matrix with uncertainties. Check your data!
INFO: 2024-11-27 15:37:19,714 - ProMCDA - Either standard deviation values of normal/lognormal indicators are larger than their means
INFO: 2024-11-27 15:37:19,715 - ProMCDA - or max. values of uniform distributed indicators are smaller than their min. values.
INFO: 2024-11-27 15:37:19,716 - ProMCDA - If you continue, the negative values will be rescaled internally to a positive range.


'5 normalized matrices have been normalized.'

In [11]:
promcda.normalize(Normalization)

INFO: 2024-11-27 15:39:18,726 - ProMCDA - Alternatives are ['alt1', 'alt2', 'alt3', 'alt4']
INFO: 2024-11-27 15:39:18,735 - ProMCDA - There is a problem with the parameters given in the input matrix with uncertainties. Check your data!
INFO: 2024-11-27 15:39:18,736 - ProMCDA - Either standard deviation values of normal/lognormal indicators are larger than their means
INFO: 2024-11-27 15:39:18,737 - ProMCDA - or max. values of uniform distributed indicators are smaller than their min. values.
INFO: 2024-11-27 15:39:18,738 - ProMCDA - If you continue, the negative values will be rescaled internally to a positive range.
ERROR: 2024-11-27 15:39:19,851 - ProMCDA utils for parallelization - Error Message
Stack (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ProMCDA/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/ProMC

ValueError: The selected normalization method is not supported

In [ ]:
promcda.aggregate(None, None,[0.5, 0.7])

In [ ]:
promcda.aggregate("minmax", "geometric",[0.5, 0.7])